# Advanced `elif` — Writing Clear, Correct Branches

This notebook deepens the `if / elif / else` topic:
- Replacing nested `if` trees with flat, mutually exclusive branches
- Correct ordering and boundary coverage
- Detecting overlapping/unreachable branches
- Chained comparisons & containment in conditions
- Guard clauses and early returns (to avoid giant ladders)
- Data-driven alternatives to big `elif` ladders
- Exercises with tests and solutions

## 1) `elif` flattens nested trees
Any time you see an `else:` followed immediately by an `if`, think `elif`.

### Nested version (harder to scan):

In [8]:
grade = 72
if grade >= 90:
    letter = 'A'
else:
    if grade >= 80:
        letter = 'B'
    else:
        if grade >= 70:
            letter = 'C'
        else:
            if grade >= 60:
                letter = 'D'
            else:
                letter = 'F'
print(letter)  # C

C


### Equivalent `elif` version (flat & mutually exclusive):

In [9]:
grade = 72
if grade >= 90:
    letter = 'A'
elif grade >= 80:
    letter = 'B'
elif grade >= 70:
    letter = 'C'
elif grade >= 60:
    letter = 'D'
else:
    letter = 'F'
print(letter)  # C

C


## 2) Ordering and boundaries matter
Place the most restrictive/highest thresholds first. Decide **inclusive** vs **exclusive** boundaries and stick with them.

In [10]:
def bracket(n):
    # Intention: [0, 10] -> 'low', (10, 20] -> 'mid', > 20 -> 'high'
    if n <= 10:
        return 'low'
    elif n <= 20:  # at this point n > 10 holds from the previous branch
        return 'mid'
    else:
        return 'high'

    # quick checks
assert bracket(0) == 'low'
assert bracket(10) == 'low'     # inclusive lower range
assert bracket(11) == 'mid'
assert bracket(20) == 'mid'
assert bracket(21) == 'high'

### Pitfall: overlapping conditions make earlier branches shadow later ones

In [11]:
def label_bad(x):
    if x >= 0:
        return 'non-negative'
    elif x >= 10:  # unreachable: if x >= 10, first branch already returned
        return '10 or more'
    else:
        return 'negative'

assert label_bad(15) == 'non-negative'  # but maybe you wanted '10 or more'

**Fix by ordering strongest condition first**:

In [12]:
def label_good(x):
    if x >= 10:
        return '10 or more'
    elif x >= 0:
        return 'non-negative'
    else:
        return 'negative'

assert label_good(15) == '10 or more'
assert label_good(5) == 'non-negative'
assert label_good(-1) == 'negative'

## 3) Chained comparisons & containment simplify conditions

In [13]:
def temp_bucket(c):
    if c <= 0:
        return 'freezing'
    elif 1 <= c <= 10:  # chained comparison reads like math
        return 'cold'
    elif 11 <= c <= 20:
        return 'mild'
    elif 21 <= c <= 30:
        return 'warm'
    else:
        return 'hot'

assert temp_bucket(0) == 'freezing'
assert temp_bucket(10) == 'cold'
assert temp_bucket(15) == 'mild'
assert temp_bucket(25) == 'warm'
assert temp_bucket(31) == 'hot'

def transport(mode):
    if mode in {'car','van','truck'}:
        return 'road'
    elif mode in {'boat','ship'}:
        return 'water'
    elif mode in {'plane','jet'}:
        return 'air'
    else:
        return 'unknown'

assert transport('truck') == 'road'
assert transport('boat') == 'water'
assert transport('jet') == 'air'
assert transport('bike') == 'unknown'

## 4) Guard clauses reduce `elif` depth in functions
Return early for invalid states; keep the happy path short and clear.

In [14]:
def authorize_withdraw(balance, amount, enabled):
    if not enabled:
        return 'account disabled'
    if amount <= 0:
        return 'invalid amount'
    if amount > balance:
        return 'insufficient funds'
    return 'authorized'

assert authorize_withdraw(1000, 50, True) == 'authorized'
assert authorize_withdraw(1000, 5000, True) == 'insufficient funds'
assert authorize_withdraw(1000, 50, False) == 'account disabled'

## 5) Data-driven alternatives to large `elif` ladders
When the mapping is discrete (not ranges), consider a dict. For ranges, consider a small helper table and loop.

In [15]:
# Discrete mapping
SEVERITY = {
    'DEBUG': 10,
    'INFO': 20,
    'WARNING': 30,
    'ERROR': 40,
    'CRITICAL': 50,
}

def sev(level: str) -> int:
    return SEVERITY.get(level.upper(), 0)

assert sev('info') == 20
assert sev('unknown') == 0

# Range table (upper bounds)
GRADE_TABLE = [
    (59, 'F'), (69, 'D'), (79, 'C'), (89, 'B'), (100, 'A')
]

def letter_by_table(score: int) -> str:
    for upper, label in GRADE_TABLE:
        if score <= upper:
            return label
    return 'A'  # fallback for >100 (or raise)

assert letter_by_table(72) == 'C'
assert letter_by_table(89) == 'B'
assert letter_by_table(101) == 'A'

---
## 6) Exercises
Try these first, then reveal the **Solutions** cell and run the tests.

**Exercise A — Shipping bands**

Given a package weight `w` (kg), return a band:
- `w <= 1` → `'XS'`
- `1 < w <= 5` → `'S'`
- `5 < w <= 20` → `'M'`
- `20 < w <= 50` → `'L'`
- `w > 50` → `'XL'`

Use a clean `elif` ladder with correct, non-overlapping boundaries.

In [16]:
def shipping_band(w: float) -> str:
    # TODO: implement with elif, covering all weights
    raise NotImplementedError()

# Examples to pass:
# assert shipping_band(0.8) == 'XS'
# assert shipping_band(1.0) == 'XS'
# assert shipping_band(1.1) == 'S'
# assert shipping_band(5) == 'S'
# assert shipping_band(5.1) == 'M'
# assert shipping_band(20) == 'M'
# assert shipping_band(35) == 'L'
# assert shipping_band(50) == 'L'
# assert shipping_band(55) == 'XL'

**Exercise B — Score to label with chained comparisons**

Return `'low'` for `0–10`, `'mid'` for `11–20`, `'high'` for `21–30`, otherwise `'out'`.
Use chained comparisons (e.g., `11 <= n <= 20`).

In [17]:
def score_label(n: int) -> str:
    # TODO: implement using chained comparisons
    raise NotImplementedError()

# assert score_label(0) == 'low'
# assert score_label(10) == 'low'
# assert score_label(11) == 'mid'
# assert score_label(20) == 'mid'
# assert score_label(22) == 'high'
# assert score_label(31) == 'out'

**Exercise C — Overlap check (reasoning)**

Consider the following (buggy) ladder and **fix the order** so that all branches are reachable:
```python
def buggy(x):
    if x >= 0:
        return 'non-neg'
    elif x >= 10:
        return 'big'
    else:
        return 'neg'
```
Write `fixed(x)` that returns `'big'` for `x >= 10`, `'non-neg'` for `0 <= x < 10`, `'neg'` otherwise.

In [18]:
def fixed(x: int) -> str:
    # TODO: order strongest condition first
    raise NotImplementedError()

# assert fixed(15) == 'big'
# assert fixed(5) == 'non-neg'
# assert fixed(-1) == 'neg'

---
## Solutions (reveal when ready)

In [19]:
# Exercise A — solution
def shipping_band(w: float) -> str:
    if w <= 1:
        return 'XS'
    elif w <= 5:
        return 'S'
    elif w <= 20:
        return 'M'
    elif w <= 50:
        return 'L'
    else:
        return 'XL'

# Exercise B — solution (chained)
def score_label(n: int) -> str:
    if 0 <= n <= 10:
        return 'low'
    elif 11 <= n <= 20:
        return 'mid'
    elif 21 <= n <= 30:
        return 'high'
    else:
        return 'out'

# Exercise C — solution
def fixed(x: int) -> str:
    if x >= 10:
        return 'big'
    elif x >= 0:
        return 'non-neg'
    else:
        return 'neg'


## Tests — run to verify

In [20]:
# Exercise A
assert shipping_band(0.8) == 'XS'
assert shipping_band(1.0) == 'XS'
assert shipping_band(1.1) == 'S'
assert shipping_band(5.0) == 'S'
assert shipping_band(5.1) == 'M'
assert shipping_band(20.0) == 'M'
assert shipping_band(35.0) == 'L'
assert shipping_band(50.0) == 'L'
assert shipping_band(55.0) == 'XL'

# Exercise B
assert score_label(0) == 'low'
assert score_label(10) == 'low'
assert score_label(11) == 'mid'
assert score_label(20) == 'mid'
assert score_label(22) == 'high'
assert score_label(31) == 'out'

# Exercise C
assert fixed(15) == 'big'
assert fixed(5) == 'non-neg'
assert fixed(-1) == 'neg'

print('All elif-advanced checks passed ✅')

All elif-advanced checks passed ✅
